<a href="https://colab.research.google.com/github/TechTM-0/mediapipe/blob/main/Mediapipe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1.マウントしてColabからGoogleドライブにアクセスできるように設定

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


2.mediapipeインストール

In [ ]:
!pip install mediapipe

3.インストール確認

In [ ]:
import mediapipe as mp

try:
    print("MediaPipeのバージョン:", mp.__version__)
    print("MediaPipeは正常に動作しています！")
except AttributeError:
    print("MediaPipeはインポートできましたが、バージョン情報を取得できませんでした。")
except Exception as e:
    print(f"エラーが発生しました: {e}")

MediaPipeのバージョン: 0.10.21
MediaPipeは正常に動作しています！


4.姿勢推定を行い元動画にランドマークと接続線を描画　<br>
　実行前にinput_video_pathとoutput_video_pathを動画ファイルのパスに変更してください。

In [ ]:
import cv2
import mediapipe as mp
import numpy as np

# === 入力・出力ファイルの設定 ===
# Googleドライブ上のパスを指定
input_video_path = '/content/drive/MyDrive/material/material.mp4'

# Googleドライブ上のパスを指定
output_video_path = '/content/drive/MyDrive/output/hand_pose_cropped.mp4'

# === MediaPipe PoseとHandsの準備 ===
# ポーズ（骨格）検出用のライブラリをインポート。
mp_pose = mp.solutions.pose
# 手のランドマーク検出用のライブラリをインポート。
mp_hands = mp.solutions.hands
# 検出結果を描画するためのユーティリティをインポート。
mp_drawing = mp.solutions.drawing_utils

# ポーズ（骨格）検出モデルを初期化。
# static_image_mode=Falseに設定することで、動画（連続したフレーム）の処理に適したモードになります。
pose = mp_pose.Pose(static_image_mode=False)

# 手の検出モデルを初期化。
# static_image_mode=False：動画（連続したフレーム）モードで動作します。
# max_num_hands=2：同時に検出する手の数を最大2つに設定します。
# min_detection_confidence=0.3：検出の信頼度をデフォルトの0.5から0.3に下げることで、
# 小さく映っている手など、見逃しがちな手も捉えやすくなります。
hands = mp_hands.Hands(
    static_image_mode=False,
    max_num_hands=2,
    min_detection_confidence=0.3
)

# === 動画の読み込みと設定の取得 ===
# OpenCVを使って入力動画ファイルを開きます。
cap = cv2.VideoCapture(input_video_path)

# 動画ファイルが正常に開かれたかを確認します。
if not cap.isOpened():
    print(f"エラー: 入力動画 '{input_video_path}' を開けませんでした。")
    exit()

# 動画の幅、高さ、フレームレート（fps）を取得します。
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))

# === 出力動画の設定 ===
# 出力動画のコーデックを定義します。'mp4v'はMP4形式で広く使われます。
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
# 出力動画ファイルを作成します。元の動画と同じ設定（幅、高さ、fps）にします。
writer = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

print("動画処理を開始します...")

# === メインループ：動画の全フレームを処理 ===
# 動画が開いている間、無限ループでフレームを1枚ずつ読み込みます。
while cap.isOpened():
    # フレームを1枚読み込みます。
    ret, frame = cap.read()
    # フレームが読み込めなかった場合（動画の終端に達した場合など）はループを終了します。
    if not ret:
        break

    # MediaPipeの処理のために、色空間をBGRからRGBに変換します。
    image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # 最初の処理として、フレーム全体でポーズ（骨格）を検出します。
    # これにより、後で手の位置を特定するための情報（手首の座標）が得られます。
    pose_results = pose.process(image_rgb)

    # ポーズが検出された場合のみ、以下の処理を実行します。
    if pose_results.pose_landmarks:
        # ポーズ（骨格）の描画設定を定義します。
        # 線の色をオレンジ、点のサイズを小さくします。
        pose_spec = mp_drawing.DrawingSpec(color=(0, 0, 255), thickness=1, circle_radius=2)
        # 点の色を緑、サイズを少し大きくします。
        pose_dot_spec = mp_drawing.DrawingSpec(color=(0, 0, 255), thickness=2, circle_radius=2)

        # 検出されたポーズ（骨格）のランドマークと接続線を元のフレームに描画します。
        mp_drawing.draw_landmarks(
            frame,
            pose_results.pose_landmarks,
            mp_pose.POSE_CONNECTIONS,
            pose_spec,
            pose_dot_spec
        )

        # 左右の手首のランドマークをポーズの検出結果から取得します。
        left_wrist = pose_results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_WRIST]
        right_wrist = pose_results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_WRIST]

        # 左右の手首それぞれに対してループを回し、個別に手の検出と描画を行います。
        for wrist_landmark in [left_wrist, right_wrist]:
            # 手首のランドマークの信頼度が0.5より高い(手首が正しく検出できている)場合のみ処理を続行します。
            # これにより、誤検出による無駄な処理を防ぎます。
            if wrist_landmark.visibility > 0.3:
                # ランドマークの正規化された座標（0.0～1.0）です。
                # フレームの長さ(ピクセル)を掛けることで実際のピクセル座標に変換します。

                center_x = int(wrist_landmark.x * frame_width)
                center_y = int(wrist_landmark.y * frame_height)

                # 手を検出するために切り抜く領域のサイズを定義します。
                size = 200
                # 切り抜き領域の左上と右下の座標を計算します。
                # max()とmin()を使って、領域がフレームの範囲外に出ないように調整します。
                x1 = max(0, center_x - size // 2)
                y1 = max(0, center_y - size // 2)
                x2 = min(frame_width, center_x + size // 2)
                y2 = min(frame_height, center_y + size // 2)

                # 計算した座標に基づいて、元のフレームから手の領域を切り抜きます（クロッピング）。
                hand_crop = frame[y1:y2, x1:x2]

                # 切り抜いた領域が有効（サイズが0より大きい）な場合にのみ、次の処理に進みます。
                if hand_crop.size > 0:
                    # 切り抜いた画像をRGBに変換し、手のランドマークを再検出します。
                    # 小さく映っていた手が拡大されてモデルに渡されるため、検出精度が向上します。
                    hand_crop_rgb = cv2.cvtColor(hand_crop, cv2.COLOR_BGR2RGB)
                    hands_results = hands.process(hand_crop_rgb)

                    # 手が検出された場合のみ描画を行います。
                    if hands_results.multi_hand_landmarks:
                        for hand_landmarks in hands_results.multi_hand_landmarks:
                            # 手のランドマークの描画設定を定義します。
                            # 線の色を赤、点のサイズを小さくします。
                            hand_spec = mp_drawing.DrawingSpec(color=(0, 0, 255), thickness=1, circle_radius=1)
                            # 点の色をシアン、サイズを小さくします。
                            hand_dot_spec = mp_drawing.DrawingSpec(color=(0, 0, 255), thickness=2, circle_radius=2)

                            # 切り抜いた画像に、手のランドマークと接続線を描画します。
                            mp_drawing.draw_landmarks(
                                hand_crop,
                                hand_landmarks,
                                mp_hands.HAND_CONNECTIONS,
                                hand_spec,
                                hand_dot_spec
                            )

                            # ランドマークが描画された切り抜き画像を、元のフレームの正しい位置に貼り付け直します。
                            frame[y1:y2, x1:x2] = hand_crop

    # 処理済みのフレームを出力動画ファイルに書き込みます。
    writer.write(frame)

# === 後処理 ===
# 動画ファイルオブジェクトとモデルを解放し、リソースをクリーンアップします。
# これを怠ると、出力動画が破損することがあります。
cap.release()
writer.release()
pose.close()
hands.close()

print("動画処理が完了しました。")
print(f"'{output_video_path}' に動画が保存されました。")

動画処理を開始します...
動画処理が完了しました。
'/content/drive/MyDrive/output/hand_pose_cropped.mp4' に動画が保存されました。
